In [11]:
from instagrapi import Client

# Log in to Instagram
username = "theemune"
password = "chinatown1"

cl = Client()
cl.login(username, password)

# Post URL or shortcode
post_shortcode = "DFS-CMkyCtx"  # Replace with the actual shortcode of the post

# Fetch media ID from the shortcode
media_id = cl.media_id(cl.media_pk_from_url(f"https://www.instagram.com/p/{post_shortcode}/"))

# Fetch up to 800 comments
comments = cl.media_comments(media_id, amount=800)

# Print comments
print(f"Fetched {len(comments)} comments:")
for comment in comments:
    print(f"User: {comment.user.username}")
    print(f"Comment: {comment.text}")
    time.sleep(random.randint(15, 30))
    print("-" * 50)

Fetched 467 comments:
User: thebanterjeremy
Comment: Hey @costco I am very happy for yall and imma let you finish BUT you have like only one location in NYC,I think at the Bronx,and we Manhattanites don’t do the Bronx,so fix that…ok thank.Bye.
--------------------------------------------------
User: lisahall9039
Comment: I will now be getting a Costco card because of your DEI stance, congratulations on having morals and decency!
--------------------------------------------------
User: blkbutterflye
Comment: Thank you @costco
--------------------------------------------------
User: wr6st_
Comment: I appreciate y’all but bring back the combo pizza
--------------------------------------------------
User: busylifebalance
Comment: I signed up today and the workers there are sooo friendly! We were amazed at how friendly they were. The customer service at your store is absolutely AMAZING!!! I also made a call yesterday to customer service and the rep was sooo helpful.
------------------------

In [ ]:
from instagrapi import Client
import time
import random

# Log in to Instagram using proxy (dummy credentials used here)
username = "theemune"  # Replace with your Instagram username
password = "chinatown1"  # Replace with your Instagram password

cl = Client()
cl.login(username, password)

try:
    cl.login(username, password)
    print("Logged in successfully using proxy!")
except Exception as e:
    print(f"Login failed: {e}")
    exit()

# Loading profile data
profile_username = 'costco'  # Intended profile to scrape data from (in our case it's Costco)
try:
    profile = cl.user_info_by_username(profile_username)
    print(f"Username: {profile.username}")
    print(f"Followers: {profile.follower_count}")
    print(f"Following: {profile.following_count}")
    print(f"Biography: {profile.biography}")
except Exception as e:
    print(f"Failed to fetch profile data: {e}")
    exit()

# Iterate through posts to get the comments 
try:
    user_id = cl.user_id_from_username(profile_username)
    posts = cl.user_medias(user_id, 11)  # Fetching up to 10 posts (adjust as needed)
    for post in posts:
        print(f"Post Date: {post.taken_at}")
        print(f"Likes: {post.like_count}")
        print(f"Comments Count: {post.comment_count}")
        print(f"URL: https://www.instagram.com/p/{post.code}/")

        # Delay between 15 seconds to 1 min
        #time.sleep(random.randint(15, 60))  

        # Accessing all comments
        print("Comments:")
        try:
            comments = cl.media_comments(post.id)
            for comment in comments:
                print(f"- {comment.user.username}: {comment.text}")
        except Exception as e:
            print(f"Error while fetching comments: {e}")

        print("-" * 50)
except Exception as e:
    print(f"Failed to fetch posts: {e}")

## Reference for this code

https://www.bing.com/search?pglt=297&q=instagram+api+python&cvid=b8a7564d679c462fbb57f9b393cde280&gs_lcrp=EgRlZGdlKgYIAhAAGEAyBggAEEUYOTIGCAEQABhAMgYIAhAAGEAyBggDEAAYQDIGCAQQABhAMgYIBRAAGEAyBggGEAAYQDIGCAcQABhAMgYICBAAGEDSAQkxMjYxNGowajGoAgGwAgE&FORM=ANNTA1&PC=ASTS

In [ ]:
import json
from instagrapi import Client
import time
import random
import os

# Instagram credentials
username = "theemune"  # Replace with your Instagram username
password = "chinatown1"  # Replace with your Instagram password

# Initialize Instagram client
cl = Client()

In [ ]:
# File paths for checkpoint and output
checkpoint_file = "checkpoint.json"
output_file = "scraped_data.json"

In [ ]:
# Function to save checkpoint
def save_checkpoint(last_post_code, last_comment_index):
    checkpoint = {"last_post_code": last_post_code, "last_comment_index": last_comment_index}
    with open(checkpoint_file, "w") as f:
        json.dump(checkpoint, f)

# Function to load checkpoint
def load_checkpoint():
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, "r") as f:
            return json.load(f)
    return {"last_post_code": None, "last_comment_index": 0}

In [ ]:
# Function to save scraped data incrementally
def save_scraped_data(data):
    if os.path.exists(output_file):
        with open(output_file, "r") as f:
            existing_data = json.load(f)
    else:
        existing_data = []
    existing_data.extend(data)
    with open(output_file, "w") as f:
        json.dump(existing_data, f, indent=4)

In [ ]:
# Function to login or reconnect
def ensure_login():
    try:
        cl.login(username, password)
        print("Logged in successfully!")
    except Exception as e:
        print(f"Login failed: {e}")
        exit()

# Fetch posts from a profile
def fetch_posts(profile_username):
    try:
        user_id = cl.user_id_from_username(profile_username)
        return cl.user_medias(user_id, 50)  # Fetch up to 50 posts
    except Exception as e:
        print(f"Failed to fetch posts: {e}")
        exit()

In [ ]:
# Main execution
profile_username = "costco"
ensure_login()

# Load checkpoint
checkpoint = load_checkpoint()
last_post_code = checkpoint.get("last_post_code")
last_comment_index = checkpoint.get("last_comment_index")

# Fetch posts
posts = fetch_posts(profile_username)
resuming = False
scraped_data = []

# Loop through posts
for post in posts:
    # Resume from checkpoint
    if not resuming and last_post_code and post.code != last_post_code:
        continue
    resuming = True

    print(f"Processing Post: {post.code}")
    print(f"Likes: {post.like_count}, Comments: {post.comment_count}")
    print(f"URL: https://www.instagram.com/p/{post.code}/")

    post_data = {
        "post_code": post.code,
        "likes": post.like_count,
        "comments_count": post.comment_count,
        "url": f"https://www.instagram.com/p/{post.code}/",
        "comments": [],
    }

    try:
        comments = cl.media_comments(post.id)
        for idx, comment in enumerate(comments[last_comment_index:], start=last_comment_index):
            comment_data = {"username": comment.user.username, "text": comment.text}
            post_data["comments"].append(comment_data)

            # Save checkpoint after every 50 comments
            if (idx + 1) % 50 == 0:
                save_checkpoint(post.code, idx + 1)
                save_scraped_data([post_data])
                print("Checkpoint saved. Pausing...")
                time.sleep(random.randint(15, 30))  # Random delay to avoid rate limits

        # Reset checkpoint for the next post
        last_comment_index = 0
        save_checkpoint(post.code, 0)
        scraped_data.append(post_data)

    except Exception as e:
        if "login_required" in str(e):
            print("Session expired. Re-logging in...")
            ensure_login()
        else:
            print(f"Error while processing post: {e}")
        break  # Exit loop to handle gracefully

    print("-" * 50)

# Save final data
save_scraped_data(scraped_data)
print("Scraping complete. All data has been saved!")